In [1]:
from photerr import LsstErrorModel
import pandas as pd
import numpy as np

In [2]:
noiseless_photometry = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_data.npy")[:1000000, :6]
sps_params = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_params.npy")[:1000000, :]
catalog = pd.DataFrame(noiseless_photometry, columns=['u', 'g', 'r', 'i', 'z', 'y'])

# remove photometry brighter than brightness cut
brightness_cut = 18
for column in ['u','g','r','i','z','y']:
    catalog = catalog.drop(catalog[catalog[column] < brightness_cut].index)

catalog

,u,g,r,i,z,y
0,35.979999,34.826707,33.827043,30.591961,29.763766,24.299834
1,20.662052,20.634097,20.163877,20.307578,20.376406,20.447159
2,27.477763,26.960422,26.893122,26.893755,26.835643,26.860594
3,36.451890,35.415322,34.327853,32.541182,31.546398,30.657177
4,18.311344,18.285075,18.000323,18.051494,18.085408,18.033818
...,...,...,...,...,...,...
999994,22.466823,21.958965,21.512976,20.850251,20.974092,20.654374
999995,40.123789,38.240046,27.177094,24.436528,23.433175,22.784754
999997,28.242125,23.668817,22.560040,21.593582,20.983335,20.537410
999998,43.036269,37.746546,29.504046,27.274358,26.449181,26.099206


In [3]:
random_state=42
errModel = LsstErrorModel(sigLim=0, absFlux=True)
sig5detections = LsstErrorModel(sigLim=5)
sig2detections = LsstErrorModel(sigLim=2)

observed_catalog = errModel(catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)

catalog_sig5 = sig5detections(observed_catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig5.rename(columns={"u": "u5", "g": "g5", "r": "r5", "i": "i5", "z": "z5", "y": "y5" }, inplace=True)

catalog_sig2 = sig2detections(observed_catalog, random_state=random_state).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig2.rename(columns={"u": "u2", "g": "g2", "r": "r2", "i": "i2", "z": "z2", "y": "y2" }, inplace=True)

observed_catalog = observed_catalog.join(catalog_sig5).join(catalog_sig2)
observed_catalog.dropna(axis=0, subset=['i5'], inplace=True) #require 5sigma detection in i band

In [4]:
observed_catalog

,u,g,r,i,z,y,u5,g5,r5,i5,z5,y5,u2,g2,r2,i2,z2,y2
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743,20.660658,20.637301,20.164046,20.316226,20.367368,20.438331,20.660658,20.637301,20.164046,20.316226,20.367368,20.438331
4,18.313505,18.286843,17.997658,18.049663,18.083339,18.031653,18.315667,18.288610,17.994992,18.047832,18.081270,18.029488,18.315667,18.288610,17.994992,18.047832,18.081270,18.029488
9,28.607734,29.551780,26.678613,25.269228,24.819414,24.586305,NaN,NaN,26.712498,25.196724,24.872299,24.482306,NaN,NaN,26.712498,25.196724,24.872299,24.482306
11,22.894148,22.003180,20.552170,21.234631,20.002153,21.202984,22.917348,22.004981,20.551348,21.231585,19.998538,21.197273,22.917348,22.004981,20.551348,21.231585,19.998538,21.197273
13,27.962640,28.348862,27.689416,27.014244,26.836585,26.212548,NaN,NaN,NaN,26.855108,NaN,NaN,NaN,28.096001,27.646537,26.855108,27.012271,26.135618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999993,28.796676,27.730703,26.663461,26.006994,24.442110,24.057591,NaN,NaN,26.595126,26.059789,24.444836,24.046818,NaN,27.695457,26.595126,26.059789,24.444836,24.046818
999994,22.466220,21.953191,21.509219,20.848127,20.973475,20.644407,22.465617,21.947420,21.505462,20.846003,20.972858,20.634464,22.465617,21.947420,21.505462,20.846003,20.972858,20.634464
999995,27.151920,31.265168,26.918491,24.452270,23.404755,22.808268,NaN,NaN,26.709695,24.468231,23.376991,22.832271,NaN,NaN,26.709695,24.468231,23.376991,22.832271
999997,26.879317,23.672854,22.564309,21.596926,20.978265,20.522467,NaN,23.676901,22.568582,21.600271,20.973197,20.507571,26.293134,23.676901,22.568582,21.600271,20.973197,20.507571


In [7]:
def select_u_dropouts(observed_catalog):
    
    #observed_catalog.dropna(axis=0, subset=['g5'], inplace=True)
    udrop = observed_catalog.drop(observed_catalog[np.isnan(observed_catalog.u2) == False].index).filter(['u','g','r','i','z','y'])

    return udrop

def select_g_dropouts(observed_catalog):
    
    #observed_catalog.dropna(axis=0, subset=['r5'], inplace=True)
    gdrop = observed_catalog.drop(observed_catalog[np.isnan(observed_catalog.g2) == False].index)
    #gdrop = gdrop.drop(gdrop[np.isnan(gdrop.u2) == False].index).filter(['u','g','r','i','z','y'])
    
    return gdrop

def select_r_dropouts(observed_catalog):
    
    #observed_catalog.dropna(axis=0, subset=['i5'], inplace=True)
    rdrop = observed_catalog.drop(observed_catalog[np.isnan(observed_catalog.r2) == False].index)
    #rdrop = rdrop.drop(rdrop[np.isnan(rdrop.g2) == False].index).filter(['u','g','r','i','z','y'])
    
    return rdrop

In [8]:
select_r_dropouts(observed_catalog)

,u,g,r,i,z,y,u5,g5,r5,i5,z5,y5,u2,g2,r2,i2,z2,y2
27,30.001641,29.062804,28.798265,25.147594,24.587191,24.145779,NaN,NaN,NaN,25.202168,24.634805,24.201802,NaN,NaN,NaN,25.202168,24.634805,24.201802
69,28.110507,29.256440,29.309333,27.017322,25.007341,24.925844,NaN,NaN,NaN,26.825187,25.006532,24.738399,NaN,NaN,NaN,26.825187,25.006532,24.738399
91,29.147280,29.899721,30.567273,26.249320,23.522908,21.931051,NaN,NaN,NaN,26.140045,23.557947,21.934798,NaN,NaN,NaN,26.140045,23.557947,21.934798
96,27.339124,31.534869,28.401335,23.938456,22.465748,22.035339,NaN,NaN,NaN,23.939802,22.458970,22.031200,NaN,NaN,NaN,23.939802,22.458970,22.031200
111,29.359089,28.399742,28.516022,26.765913,26.024836,27.257653,NaN,NaN,NaN,26.717802,25.738456,NaN,NaN,27.708245,NaN,26.717802,25.738456,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999884,29.782539,29.329862,28.566761,21.537337,20.713144,19.410356,NaN,NaN,NaN,21.545769,20.714470,19.411084,NaN,NaN,NaN,21.545769,20.714470,19.411084
999907,29.653881,29.231136,27.802255,23.775323,22.890952,22.068503,NaN,NaN,NaN,23.783922,22.886614,22.066339,NaN,NaN,NaN,23.783922,22.886614,22.066339
999912,28.387533,31.645695,29.296311,23.619669,22.599165,21.676583,NaN,NaN,NaN,23.625684,22.594052,21.676929,NaN,NaN,NaN,23.625684,22.594052,21.676929
999959,29.746468,30.177542,30.977223,26.563819,24.682676,22.049621,NaN,NaN,NaN,26.828443,24.707066,22.050976,NaN,NaN,NaN,26.828443,24.707066,22.050976
